In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [2]:
client = MongoClient()
characters = client.ck2.characters

## Descendents of Charlemagne

Same calculation as in notebook 2 but now including all descendents through female descendents as well by using the parents array ([fat, mot]) instead of just the fat value for connectToField. Over the 26 generations found in notebook 2 there are 48,000 characters that claim descent from Charlemagne. 

In [3]:
pipeline = [
    {
        "$match" : {"_id" : 6392}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "parents",
            "as": "descendent"
        }},
    {"$project" : { "bn" : 1, "descendents" : {"$size" : "$descendent"} }    }
]

In [4]:
karl = characters.aggregate(pipeline)

In [5]:
for char in karl:
    print(char)

{'_id': 6392, 'bn': 'Charles', 'descendents': 48501}


## Groupby Family

Most marriages by Charlemagne's descendents have been into the Alachisling dynasty. The Isauros rule Byzantium for a while before eventually losing power to the Kyritzios. The Uuidoch rule Scotland and briefly a crusade liberated Greece while the Penikis rise to power in Bulgaria, Finland and Russia. Charlemagne has many descendents and they marry into many of the most powerful houses in Europe.

A list of all families to rule a Kingdom or Empire level title and the titles they held is shown in notebook 7.1 Titles per Dynasty.

In [6]:
pipeline = [
    {
        "$match" : {"_id" : 6392}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "parents",
            "as": "descendent",
            "depthField": "generation"
            #"maxDepth" : 1
        }
    },    
    {"$unwind" : "$descendent" },
    {"$lookup" :
     {
            "from" : "dynasties",
            "localField" : "descendent.dnt",
            "foreignField" : "_id",
            "as" : "descendent_dyn"
        }
    },
    {"$unwind" : "$descendent_dyn" },
    {
        "$group":
        {
            "_id" : "$descendent_dyn.name",
            "total" : {"$sum" : 1}
        }
    },
    {"$sort" : { "total" : -1 } },
    {"$limit" : 10}
]

In [7]:
karl = characters.aggregate(pipeline)
for char in karl:
    print(char)

{'_id': 'Alachisling', 'total': 728}
{'_id': 'Isauros', 'total': 613}
{'_id': 'Penikis', 'total': 506}
{'_id': 'Abbasid', 'total': 488}
{'_id': 'Uuidoch', 'total': 485}
{'_id': 'Rangabes', 'total': 465}
{'_id': 'Makrembolites', 'total': 451}
{'_id': 'Palemonaitis', 'total': 447}
{'_id': 'Kyritzios', 'total': 439}
{'_id': 'Uzur', 'total': 366}


## Real Descent

This cell uses the rparents value. Thus the rfat value will be used if it is set, if not the fat value will be set. The rfat value will be set where the character was born out of wedlock or their stated father isn't their real father. This calculation will drop any descendents, and their descendents, where their real father isn't a descendent of Charlemagne. It will include children who are descendents but are illegitimate don't know their true heritage.

In [8]:
pipeline = [
    {
        "$match" : {"_id" : 6392}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "rparents",
            "as": "descendent"
        }},
    {"$project" : { "bn" : 1, "descendents" : {"$size" : "$descendent"} }    }
]

In [9]:
for char in characters.aggregate(pipeline):
    print(char)

{'_id': 6392, 'bn': 'Charles', 'descendents': 49216}


# Real GroupBy Family

In [10]:
pipeline = [
    {
        "$match" : {"_id" : 6392}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "rparents",
            "as": "descendent",
            "depthField": "generation"
        }},    
    {"$unwind" : "$descendent" },
    {"$lookup" :
     {
            "from" : "dynasties",
            "localField" : "descendent.dnt",
            "foreignField" : "_id",
            "as" : "descendent_dyn"
        }
    },
    {"$unwind" : "$descendent_dyn" },
    {
        "$group":
        {
            "_id" : "$descendent_dyn.name",
            "total" : {"$sum" : 1}
        }
    },
    {"$sort" : { "total" : -1 } },
    {"$limit" : 10}
]

In [11]:
karl = characters.aggregate(pipeline)
for char in karl:
    print(char)

{'_id': 'Alachisling', 'total': 715}
{'_id': 'Isauros', 'total': 610}
{'_id': 'Penikis', 'total': 498}
{'_id': 'Uuidoch', 'total': 487}
{'_id': 'Abbasid', 'total': 478}
{'_id': 'Rangabes', 'total': 459}
{'_id': 'Makrembolites', 'total': 450}
{'_id': 'Palemonaitis', 'total': 446}
{'_id': 'Kyritzios', 'total': 428}
{'_id': 'Kritopolos', 'total': 361}


# Shared Descendents

This is something I looked at only quickly. I take two characters, in this case Charlemagne and Cobthach Eóganacht-Locha Léin, Count of Thomond, died 789, and find all descendents that they have in common. In total Cobhtach has 14000 descendents so the two lines mearge early on. Both men share over 10000 descendents in common. Most of those descendents are members of the Scottish Uuidoch dynasty, but also of the Mel dynasty that forms England and the de Chaumontois rulers of Lotharingia.

In [12]:
pipeline = [
    {
        "$match" : {"_id" : 6392} #Charlemagne
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "parents",
            "as": "descendent"
        }
    },
    {
        "$unwind" : "$descendent"
    },
    {
        "$project" : { "_id" : 0, "charId" : "$descendent._id" }
    }
]

In [13]:
char1 = set()

for karl in characters.aggregate(pipeline):
    char1.add(karl["charId"])

In [14]:
pipeline = [
    {
        "$match" : {"_id" : 83791} #Cobthach Count of Thomond 745-789
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "parents",
            "as": "descendent"
        }
    },
    {
        "$unwind" : "$descendent"
    },
    {
        "$project" : { "_id" : 0, "charId" : "$descendent._id" }
    }
]

In [15]:
char2 = set()

for karl in characters.aggregate(pipeline):
    char2.add(karl["charId"])

In [16]:
comb = char1.intersection(char2)
len(comb)

10414

In [17]:
pipeline = [
    {
        "$match" : {"_id" : {"$in" : list(comb)}}
    },      
    {
        "$lookup": 
        {
            "from": "dynasties",
            "localField": "dnt",
            "foreignField": "_id",
            "as": "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    {
       "$group" : { "_id" : "$dynasty._id", "name" : {"$first" : "$dynasty.name"}, "total" : {"$sum" : 1} }
    },
    {
        "$sort" : {"total" : -1}
    }
]

In [18]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 1040016, 'name': 'Uuidoch', 'total': 473}
{'_id': 25060, 'name': 'de Chaumontois', 'total': 263}
{'_id': 1044053, 'name': 'Alachisling', 'total': 256}
{'_id': 10295042, 'name': 'Mel', 'total': 221}
{'_id': 1044115, 'name': 'Abbonid', 'total': 201}
{'_id': 25061, 'name': 'Karling', 'total': 173}
{'_id': 1044116, 'name': 'Nibelunging', 'total': 150}
{'_id': 623, 'name': 'de Lagery', 'total': 144}
{'_id': 9230, 'name': 'Eóganacht-Locha Léin', 'total': 142}
{'_id': 544, 'name': 'de Soissons', 'total': 137}
{'_id': 356, 'name': 'de Baalun', 'total': 115}
{'_id': 306, 'name': 'du Perche', 'total': 111}
{'_id': 1044100, 'name': 'Emichid', 'total': 109}
{'_id': 9231, 'name': 'Mac Finnachta', 'total': 109}
{'_id': 10305082, 'name': 'Adalbertingi', 'total': 101}
{'_id': 10308289, 'name': 'de Trier', 'total': 98}
{'_id': 10297465, 'name': 'Falco', 'total': 97}
{'_id': 1044084, 'name': 'Liutprandingi', 'total': 92}
{'_id': 360, 'name': 'Estouteville', 'total': 90}
{'_id': 10302654, 'name':

{'_id': 10308448, 'name': 'Richfriding', 'total': 1}
{'_id': 10304974, 'name': 'Clwyd', 'total': 1}
{'_id': 10304375, 'name': 'de Oca', 'total': 1}
{'_id': 100374, 'name': 'Goštautas', 'total': 1}
{'_id': 10310627, 'name': 'Strathearn', 'total': 1}
{'_id': 10312466, 'name': 'of Dorset', 'total': 1}
{'_id': 10314408, 'name': 'de Saintonge', 'total': 1}
{'_id': 10313896, 'name': 'Giselbertling', 'total': 1}
{'_id': 10296168, 'name': 'de Lyon', 'total': 1}
{'_id': 10304291, 'name': 'Ua Cruadláich', 'total': 1}
{'_id': 10307017, 'name': 'of Westmorland', 'total': 1}
{'_id': 10313144, 'name': 'Bontemps', 'total': 1}
{'_id': 10308293, 'name': 'of Ternyllwg', 'total': 1}
{'_id': 10308288, 'name': 'van Périgord', 'total': 1}
{'_id': 10314526, 'name': 'of Korinthos', 'total': 1}
{'_id': 10320134, 'name': 'of Wessex', 'total': 1}
{'_id': 10316825, 'name': 'of Ulster', 'total': 1}
{'_id': 10316842, 'name': 'de Bihar', 'total': 1}
{'_id': 10314712, 'name': 'des Roches', 'total': 1}
{'_id': 1031337

# Descent from Muhammad Aleppo

In the next notebook Muhammad Aleppo (died 994) will show up with the record number of children. They won't aid him in securing a legacy however as he has only 3435 descendents in total by the end of the game. As many are illegitimate this may have made it difficult to find spouses and indeed there is a high number of murders, 326 of them are killed. Both the Sami and the Muradid dynasties will rule Persia but there are no other big name title holding dynasties in the top 10 that are descended from him. 

In [19]:
pipeline = [
    {
        "$match" : {"_id" : 629398}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "rparents",
            "as": "descendent"
        }},
    {"$project" : { "bn" : 1, "descendents" : {"$size" : "$descendent"} }    }
]

In [20]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 629398, 'bn': 'Muhammad', 'descendents': 3435}


## Dynasties Descended from Muhammad Aleppo

In [21]:
pipeline = [
    {
        "$match" : {"_id" : 629398}
    },      
    {
        "$graphLookup": 
        {
            "from": "characters",
            "startWith": "$_id",
            "connectFromField": "_id",
            "connectToField": "rparents",
            "as": "descendent",
            "depthField": "generation"
        }},    
    {"$unwind" : "$descendent" },
    {"$lookup" :
     {
            "from" : "dynasties",
            "localField" : "descendent.dnt",
            "foreignField" : "_id",
            "as" : "descendent_dyn"
        }
    },
    {"$unwind" : "$descendent_dyn" },
    {
        "$group":
        {
            "_id" : "$descendent_dyn.name",
            "total" : {"$sum" : 1}
        }
    },
    {"$sort" : { "total" : -1 } },
    {"$limit" : 10}
]

In [22]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 'Amrubid', 'total': 396}
{'_id': 'Sami', 'total': 299}
{'_id': 'Khaireddin', 'total': 152}
{'_id': 'Aligoodarzid', 'total': 139}
{'_id': 'Muradid', 'total': 132}
{'_id': 'Ali', 'total': 120}
{'_id': 'Karen', 'total': 112}
{'_id': 'Tatikid', 'total': 111}
{'_id': 'Sajrid', 'total': 103}
{'_id': 'Shujahid', 'total': 97}


## Descendents who were Killed

In [23]:
pipeline = [   
    {
    
    "$match" : { "_id" : 629398}
    },
    {
    
      "$graphLookup": {
      "from": "characters",
      "startWith": "$_id",
      "connectFromField": "_id",
      "connectToField": "rparents",
      "as": "descendent",
      "depthField": "generation"
   }},
    {"$unwind" : "$descendent"},
    {"$sort" : { "descendent.b_d": -1 }},
    {"$match" : {"descendent.killer" : {"$exists" : True}}},
    #{"$project" : {"_id" : 0, "descendent" : 1}}
    {"$group" : {"_id" : None, "total" : {"$sum" : 1}}}
]

In [24]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': None, 'total': 326}
